# Implementacja w Pythonie

Poniżej przedstawiam implementację metody zbiorów odniesienia dla problemu ciągłego 4-kryterialnego z dwoma zmiennymi decyzyjnymi oraz dla problemów dyskretnych 3- i 4-kryterialnych.

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Definicja funkcji celu
def f1(x):
    return x[0]**2 + (x[1]+0.2)**2

def f2(x):
    return (x[0]-1)**2 + (x[1]+1)**2

def f3(x):
    return (x[0] - 3)**2 + (x[1] - 4)**2

def f4(x):
    return (x[0] - 4)**2 + (x[1] - 3)**2

# Funkcja agregująca wartości funkcji celu
def F(x):
    return np.array([f1(x), f2(x), f3(x), f4(x)])

# Punkty odniesienia
a = np.array([0.0, 0.0, 0.0, 0.0])  # Punkt aspiracji (pożądany)
b = np.array([10.0, 10.0, 10.0, 10.0])  # Punkt status quo (niepożądany)

# Funkcja skoringowa oparta na metodzie TOPSIS
def scoring_function(x, a, b):
    f_values = F(x)
    # Normalizacja
    f_norm = (f_values - a) / (b - a)
    # Obliczanie odległości od punktu aspiracji i status quo
    distance_to_a = np.linalg.norm(f_norm - np.zeros(len(f_norm)))
    distance_to_b = np.linalg.norm(f_norm - np.ones(len(f_norm)))
    # Wskaźnik bliskości do ideału
    C = distance_to_b / (distance_to_a + distance_to_b)
    return C

# Funkcja do minimalizacji (negacja wskaźnika C)
def objective(x):
    return -scoring_function(x, a, b)

# Ograniczenia zmiennych decyzyjnych
bounds = [(-5, 10), (-5, 10)]

# Optymalizacja
result = minimize(
    objective,
    x0=np.array([0.0, 0.0]),  # Punkt startowy
    bounds=bounds,
    method='SLSQP'
)

optimal_x = result.x
optimal_f_values = F(optimal_x)
optimal_score = scoring_function(optimal_x, a, b)

print("Optymalne rozwiązanie:")
print(f"x = [{optimal_x[0]:.4f}, {optimal_x[1]:.4f}]")
print("Wartości funkcji celu:")
print(f"f1(x) = {optimal_f_values[0]:.4f}")
print(f"f2(x) = {optimal_f_values[1]:.4f}")
print(f"f3(x) = {optimal_f_values[2]:.4f}")
print(f"f4(x) = {optimal_f_values[3]:.4f}")
print(f"Wartość funkcji skoringowej: {optimal_score:.4f}")


Optymalne rozwiązanie:
x = [-5.0000, -5.0000]
Wartości funkcji celu:
f1(x) = 85.0000
f2(x) = 85.0000
f3(x) = 145.0000
f4(x) = 145.0000
Wartość funkcji skoringowej: 0.4788


## Wyjaśnienie:

- Funkcje celu: Zdefiniowano cztery funkcje celu 𝑓1, 𝑓2, 𝑓3, 𝑓4
- Punkty odniesienia: Punkt aspiracji 𝑎 i punkt status quo 𝑏 określają preferencje decydenta.
- Funkcja skoringowa: Bazuje na metodzie TOPSIS, uwzględniając odległości od punktów odniesienia.
- Optymalizacja: Celem jest maksymalizacja wskaźnika 𝐶, więc minimalizujemy jego negację.
- Wynik: Otrzymujemy optymalne wartości zmiennych decyzyjnych oraz odpowiadające im wartości funkcji celu i funkcji skoringowej.

# Problem Dyskretny: 3-Kryterialny

Mamy zbiór dyskretnych alternatyw ocenianych według trzech kryteriów. Naszym celem jest uszeregowanie tych alternatyw na podstawie funkcji skoringowej.

In [7]:
import numpy as np

# Lista alternatyw (rozwiązań)
alternatives = np.array([
    [5, 7, 6],
    [3, 8, 5],
    [6, 6, 7],
    [2, 9, 4],
    [4, 5, 8],
    [7, 4, 5],
    [5, 6, 6],
    [6, 5, 7]
])

# Punkty odniesienia
a = np.array([7, 9, 8])  # Punkt aspiracji
b = np.array([2, 4, 4])  # Punkt status quo

# Normalizacja alternatyw
def normalize(alternatives, a, b):
    return (alternatives - b) / (a - b)

# Funkcja skoringowa
def scoring_function(alternatives, a, b):
    norm_alt = normalize(alternatives, a, b)
    distance_to_a = np.linalg.norm(norm_alt - np.ones((len(alternatives), len(a))), axis=1)
    distance_to_b = np.linalg.norm(norm_alt - np.zeros((len(alternatives), len(a))), axis=1)
    C = distance_to_b / (distance_to_a + distance_to_b)
    return C

# Obliczanie wartości funkcji skoringowej
scores = scoring_function(alternatives, a, b)

# Tworzenie rankingu
ranking = np.argsort(-scores)  # Negacja, bo sortujemy malejąco

print("Ranking alternatyw:")
for idx in ranking:
    print(f"Alternatywa {idx + 1}: Kryteria = {alternatives[idx]}, Score = {scores[idx]:.4f}")


Ranking alternatyw:
Alternatywa 3: Kryteria = [6 6 7], Score = 0.6319
Alternatywa 1: Kryteria = [5 7 6], Score = 0.5661
Alternatywa 8: Kryteria = [6 5 7], Score = 0.5640
Alternatywa 5: Kryteria = [4 5 8], Score = 0.5228
Alternatywa 7: Kryteria = [5 6 6], Score = 0.5000
Alternatywa 6: Kryteria = [7 4 5], Score = 0.4519
Alternatywa 2: Kryteria = [3 8 5], Score = 0.4360
Alternatywa 4: Kryteria = [2 9 4], Score = 0.4142


## Wyjaśnienie:

- Alternatywy: Zbiór dyskretnych rozwiązań ocenianych według trzech kryteriów.
- Normalizacja: Przekształcamy wartości kryteriów do przedziału [0,1] na podstawie punktów odniesienia.
- Funkcja skoringowa: Obliczamy wskaźnik 𝐶 dla każdej alternatywy.
- Ranking: Sortujemy alternatywy według wartości wskaźnika 𝐶.

# Problem Dyskretny: 4-Kryterialny

In [8]:
import numpy as np

# Lista alternatyw (rozwiązań)
alternatives = np.array([
    [5, 7, 6, 5],
    [3, 8, 5, 6],
    [6, 6, 7, 7],
    [2, 9, 4, 5],
    [4, 5, 8, 6],
    [7, 4, 5, 8],
    [5, 6, 6, 7],
    [6, 5, 7, 6]
])

# Punkty odniesienia
a = np.array([7, 9, 8, 8])  # Punkt aspiracji
b = np.array([2, 4, 4, 5])  # Punkt status quo

# Normalizacja alternatyw
def normalize(alternatives, a, b):
    return (alternatives - b) / (a - b)

# Funkcja skoringowa
def scoring_function(alternatives, a, b):
    norm_alt = normalize(alternatives, a, b)
    distance_to_a = np.linalg.norm(norm_alt - np.ones((len(alternatives), len(a))), axis=1)
    distance_to_b = np.linalg.norm(norm_alt - np.zeros((len(alternatives), len(a))), axis=1)
    C = distance_to_b / (distance_to_a + distance_to_b)
    return C

# Obliczanie wartości funkcji skoringowej
scores = scoring_function(alternatives, a, b)

# Tworzenie rankingu
ranking = np.argsort(-scores)  # Negacja, bo sortujemy malejąco

print("Ranking alternatyw:")
for idx in ranking:
    print(f"Alternatywa {idx + 1}: Kryteria = {alternatives[idx]}, Score = {scores[idx]:.4f}")


Ranking alternatyw:
Alternatywa 3: Kryteria = [6 6 7 7], Score = 0.6396
Alternatywa 7: Kryteria = [5 6 6 7], Score = 0.5400
Alternatywa 6: Kryteria = [7 4 5 8], Score = 0.5346
Alternatywa 8: Kryteria = [6 5 7 6], Score = 0.5164
Alternatywa 5: Kryteria = [4 5 8 6], Score = 0.4879
Alternatywa 1: Kryteria = [5 7 6 5], Score = 0.4401
Alternatywa 2: Kryteria = [3 8 5 6], Score = 0.4157
Alternatywa 4: Kryteria = [2 9 4 5], Score = 0.3660


## Wyjaśnienie:

- Dodatkowe Kryterium: Uwzględniamy czwarte kryterium w ocenie alternatyw.
- Funkcja Skoringowa: Analogicznie do przypadku 3-kryterialnego, ale z uwzględnieniem dodatkowego kryterium.
- Ranking: Otrzymujemy uszeregowanie alternatyw według zaktualizowanej funkcji skoringowej.